In [23]:
%%clingo -n 0

x :- y. % I cannot deduce if `y` is true, so I keep it as false.
y :- x. % Same thing for `x`.

z :- not not w. % I can accept `w` for being true as long as it is coherent.
w :- not not z. % Same thing for `z`.

clingo version 5.4.0
Reading from stdin
Solving...
Answer: 1

Answer: 2
z w
SATISFIABLE

Models       : 2
Calls        : 1
Time         : 0.000s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.000s


In [24]:
%%clingo -n 0

vertex(a;b;c;d;e).

% Observed labels.
obs_e_label(a,e,1;b,e,1;c,e,1;d,e,1).
obs_v_label(a,0;b,0;c,0;d,0).

sign(1;-1).

% Mesurements in 5 levels.
obs(1;-1;0;notPlus;notMinus).

% Regular aliases.
vertex(V) :- obs_v_label(V,S).
edge(X,Y) :- obs_e_label(X,Y,S).
vertex(X) :- edge(X,Y).
vertex(Y) :- edge(X,Y).
 
% for each vertex the measurements are either changing (1,-1) or not (0)
% ? THERE IS A BUG? SHOULD BE `1{..}1`?
1 {vlabel(V,1); vlabel(V,-1); vlabel(V,0)} :- vertex(V).
% for each edge, it can get only one label.
1 {elabel(U,V,1); elabel(U,V,-1)} 1 :- edge(U,V).

% keep observed labeling of the edges
error_edge(U,V) :- obs_e_label(U,V,1), obs_e_label(U,V,-1).
elabel(U,V,S) :- edge(U,V), obs_e_label(U,V,S), not error_edge(U,V).

% influences
infl(V,S*T) :- elabel(U,V,S), vlabel(U,T).
% effects of a repair
infl(V,S) :- new_influence(V,S).

% pure influences
pinfl(V, 1) :- elabel(U,V, 1), vlabel(U, 1), not vlabel(U,0), not vlabel(U,-1).
pinfl(V,-1) :- elabel(U,V,-1), vlabel(U, 1), not vlabel(U,0), not vlabel(U,-1).
pinfl(V,-1) :- elabel(U,V, 1), vlabel(U,-1), not vlabel(U,0), not vlabel(U, 1).
pinfl(V, 1) :- elabel(U,V,-1), vlabel(U,-1), not vlabel(U,0), not vlabel(U, 1).
% effects of a repair
pinfl(V,S) :- new_influence(V,S).

% constraints
% ! CLEVER ALIAS!
:- forbidden(V,S), vlabel(V,S).

% measured variations must be explained by predecessor
forbidden(V, 1) :- vertex(V), not infl(V, 1).
forbidden(V,-1) :- vertex(V), not infl(V,-1).

% ? FORBID MULTIPLE INSTANCES OF `vlabel` WITH DIFFERENT SIGNS... BUT WHY NOW?
forbidden(V,S) :- vlabel(V,T), sign(S), S!=T.

% propagate forward
vlabel(V, 0) :- vertex(V), not forbidden(V, 0).
vlabel(V, 1) :- infl(V, 1), not forbidden(V, 1).
vlabel(V,-1) :- infl(V,-1), not forbidden(V,-1).

% constraint zero for or nodes
forbidden(V,0) :- pinfl(V, 1), not infl(V,-1). 
forbidden(V,0) :- pinfl(V,-1), not infl(V, 1).

% repair model
% define possible repair operations
% ? NEED EXPLANATION
new_influence(X,1) :- not not new_influence(X,1), not new_influence(X,-1), vertex(X).
new_influence(X,-1) :- not not new_influence(X,-1), not new_influence(X,1), vertex(X).

% keep observed variations
forbidden(V,T) :- obs_v_label(V,S), sign(S), sign(T), S!=T.

% A weak vertex variation has been observed
forbidden(V, 1) :- vertex(V), obs_v_label(V,notPlus).
forbidden(V,-1) :- vertex(V), obs_v_label(V,notMinus).

pred(X, 1) :- vlabel(X, 1).
pred(X,-1) :- vlabel(X,-1).
pred(X, 0) :- vlabel(X, 0).


% further rules for predicting signs

pred(V,notPlus) :- vlabel(V, 0).
pred(V,notPlus) :- vlabel(V,-1).


pred(V,notMinus):- vlabel(V, 0).
pred(V,notMinus):- vlabel(V, 1).


pred(V,change) :- vlabel(V, 1).
pred(V,change) :- vlabel(V,-1) .

% ? NEED EXPLANATION
#minimize{
           1,(X,S)  : new_influence(X,S)
         }.

#show pred/2.

clingo version 5.4.0
Reading from stdin
Solving...
Answer: 1
pred(a,change) pred(b,change) pred(e,change) pred(c,notMinus) pred(d,notMinus) pred(a,notPlus) pred(b,notPlus) pred(c,notPlus) pred(d,notPlus) pred(e,notPlus) pred(c,0) pred(d,0) pred(a,-1) pred(b,-1) pred(e,-1)
Optimization: 3
Answer: 2
pred(a,change) pred(b,change) pred(e,change) pred(c,notMinus) pred(d,notMinus) pred(a,notPlus) pred(b,notPlus) pred(c,notPlus) pred(d,notPlus) pred(e,notPlus) pred(c,0) pred(d,0) pred(a,-1) pred(b,-1) pred(e,-1)
Optimization: 2
Answer: 3
pred(b,change) pred(e,change) pred(a,notMinus) pred(c,notMinus) pred(d,notMinus) pred(a,notPlus) pred(b,notPlus) pred(c,notPlus) pred(d,notPlus) pred(e,notPlus) pred(a,0) pred(c,0) pred(d,0) pred(b,-1) pred(e,-1)
Optimization: 1
Answer: 4
pred(a,notMinus) pred(b,notMinus) pred(c,notMinus) pred(d,notMinus) pred(e,notMinus) pred(a,notPlus) pred(b,notPlus) pred(c,notPlus) pred(d,notPlus) pred(e,notPlus) pred(a,0) pred(b,0) pred(c,0) pred(d,0) pred(e,0)
Optimizat